In [7]:
import json
import os
import numpy as np
import pandas as pd
import pygmo as pg
from pygmo import hypervolume

fol = "ExpMOEAD_Neig"

In [8]:
parent_folder_path = '../EvolutionaryFramework/results/MOEA_D/' + fol

target_folders = [folder for folder in os.listdir(parent_folder_path) if folder.startswith('2023_08')]

In [10]:
folder_averages = {}

metaCriteriaBool = False


for folder in target_folders:
    folder_path = os.path.join(parent_folder_path, folder)
    files = os.listdir(folder_path)

    metaBool = False
    mutation = 0
    crossover = 0
    keyType = ""
    criteria = []
    result = [] 

    for file in files:
        if file.endswith('.json'):
            file_path = os.path.join(folder_path, file)
            with open(file_path) as f:
                parsed_json = json.load(f)

        if metaBool == False:
            n = parsed_json["metaParameters"]["numberOfNeighbours"]
            metaBool = True

        if metaCriteriaBool == False:
            for criterion in parsed_json["metaParameters"]['criteria']:
                criteria.append(criterion)
            keyType = parsed_json["metaParameters"]["melodyKeyType"]
            

        data = []
        for elem in parsed_json:
            if elem == "experiment":
                for individual in parsed_json[elem]["generation_250"]:
                    dictionary = {}
                    for criterion in criteria:
                        dictionary[criterion] = -individual['fitness'][criterion]
                    data.append(dictionary)    
        data_df = pd.DataFrame(data)
        data_df = data_df.dropna()
    
        hyper = pg.hypervolume(data_df.to_numpy())
        result.append(hyper.compute([0.0, 0.0]))

    folder_averages[keyType + " " + "generation_250 population_500 " + "n_" + str(n) + " " + folder] = (np.mean(result), np.std(result))
        


output_file = 'JSON/' + 'hyperVolume_Neig2.json'
with open(output_file, 'w') as f:
    json.dump(folder_averages, f, indent=4)

print('Folder averages saved successfully.')


Folder averages saved successfully.
